- 本身browser是非同步的動作，先完成DOM後，才下載圖片，webdriver完全無法追蹤，所以必需要有等待的策略
- 必需等待網頁的DOM的完成，webdriver才可以取得元素，但有些情況是動態網頁(javascript所建立)，所以常會遇到沒有發現這個元素(**no such element**)

- 如以下範例(必需手動建立一個html網頁)
- window被按下時,才會出現文字
```
<!doctype html>
<html>
<head>
<meta charset=utf-8>
<title>Race Condition Example</title>
</head>
<body>
    <script>
        var initialised = false;
        window.addEventListener("click", function () {
            var newElement = document.createElement("p");
            newElement.textContent = "Hello from JavaScript!";
            document.body.appendChild(newElement);
            initialised = true;
        });
    </script>
</body>
</html>

```

In [17]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
import os
#webdriver一開始會延遲等document的DOM載入，才開始find_element()，目前會找不到p元素，會出錯
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))
driver.get("file:///" + os.path.abspath("race_condition.html"))
el = driver.find_element(By.TAG_NAME, "p")
assert el.text == "Hello from JavaScript!"
print(el.text)
driver.quit()

NoSuchElementException: Message: no such element: Unable to locate element: {"method":"tag name","selector":"p"}
  (Session info: chrome=108.0.5359.94)
Stacktrace:
0   chromedriver                        0x000000010d7def38 chromedriver + 4910904
1   chromedriver                        0x000000010d75ea03 chromedriver + 4385283
2   chromedriver                        0x000000010d3a3747 chromedriver + 472903
3   chromedriver                        0x000000010d3e834c chromedriver + 754508
4   chromedriver                        0x000000010d3e85a1 chromedriver + 755105
5   chromedriver                        0x000000010d42be94 chromedriver + 1031828
6   chromedriver                        0x000000010d40e13d chromedriver + 909629
7   chromedriver                        0x000000010d42928e chromedriver + 1020558
8   chromedriver                        0x000000010d40dee3 chromedriver + 909027
9   chromedriver                        0x000000010d3d830c chromedriver + 688908
10  chromedriver      

## 明確等待時間

In [22]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
import os
from selenium.webdriver.support.wait import WebDriverWait

def document_initialised(driver):
    return driver.execute_script("return initialised")    #傳回javascript 變數 initialised

driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))
driver.get("file:///" + os.path.abspath("race_condition.html"))
WebDriverWait(driver, timeout=10).until(document_initialised)
el = driver.find_element(By.TAG_NAME, "p")
assert el.text == "Hello from JavaScript!"

In [25]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
import os
from selenium.webdriver.support.wait import WebDriverWait

driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))
driver.get("file:///" + os.path.abspath("race_condition.html"))
el = WebDriverWait(driver, timeout=10).until(lambda d: d.find_element(By.TAG_NAME,"p")) #等待p元素出現才動作
assert el.text == "Hello from JavaScript!"

## 預設所有的等待時間
- 沒有發現就等待，直到指定等待時間過期

In [26]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
import os
from selenium.webdriver.support.wait import WebDriverWait

driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))
driver.implicitly_wait(10)
driver.get("file:///" + os.path.abspath("race_condition.html"))
assert el.text == "Hello from JavaScript!"
